Install Libraries

In [1]:
!pip install geopy  
!pip install geocoder
!pip install folium 

Import libraries

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np

import requests

import json # library to handle JSON files
# uncomment this line if you haven't completed the Foursquare API lab
#!conda install -c conda-forge geopy --yes 

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder

#import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
# uncomment this line if you haven't completed the Foursquare API lab
# !conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Read csv-file by pandas (it was downloaded from https://data.london.gov.uk/dataset/land-area-and-population-density-ward-and-borough):

In [4]:
# https://data.london.gov.uk/dataset/land-area-and-population-density-ward-and-borough
ldn_df_raw = pd.read_csv("C:/Users/Juma Mosawi/Desktop/IBM Project/coursera_ibm_capstone-master/coursera_ibm_capstone-master/housing-density-borough.csv")
ldn_df_raw.head(12)

,Code,Name,Year,Source,Population,Inland_Area _Hectares,Total_Area_Hectares,Population_per_hectare,Square_Kilometres,Population_per_square_kilometre
0,E09000001,City of London,1999,ONS MYE,6581,290.4,314.9,22.7,2.9,2266.2
1,E09000001,City of London,2000,ONS MYE,7014,290.4,314.9,24.2,2.9,2415.3
2,E09000001,City of London,2001,ONS MYE,7359,290.4,314.9,25.3,2.9,2534.1
3,E09000001,City of London,2002,ONS MYE,7280,290.4,314.9,25.1,2.9,2506.9
4,E09000001,City of London,2003,ONS MYE,7115,290.4,314.9,24.5,2.9,2450.1
5,E09000001,City of London,2004,ONS MYE,7118,290.4,314.9,24.5,2.9,2451.2
6,E09000001,City of London,2005,ONS MYE,7131,290.4,314.9,24.6,2.9,2455.6
7,E09000001,City of London,2006,ONS MYE,7254,290.4,314.9,25.0,2.9,2498.0
8,E09000001,City of London,2007,ONS MYE,7607,290.4,314.9,26.2,2.9,2619.5
9,E09000001,City of London,2008,ONS MYE,7429,290.4,314.9,25.6,2.9,2558.3


Choose data for year 2020

In [5]:
ldn_2020_df = ldn_df_raw[ldn_df_raw['Year'] == 2020]
ldn_2020_df

,Code,Name,Year,Source,Population,Inland_Area _Hectares,Total_Area_Hectares,Population_per_hectare,Square_Kilometres,Population_per_square_kilometre
21,E09000001,City of London,2020,GLA Population Projections,8046,290.4,314.9,27.7,2.9,2770.7
73,E09000002,Barking and Dagenham,2020,GLA Population Projections,218367,3610.8,3779.9,60.5,36.1,6047.6
125,E09000003,Barnet,2020,GLA Population Projections,407148,8674.8,8674.8,46.9,86.7,4693.4
177,E09000004,Bexley,2020,GLA Population Projections,254970,6058.1,6428.6,42.1,60.6,4208.8
229,E09000005,Brent,2020,GLA Population Projections,343853,4323.3,4323.3,79.5,43.2,7953.6
281,E09000006,Bromley,2020,GLA Population Projections,336395,15013.5,15013.5,22.4,150.1,2240.6
333,E09000007,Camden,2020,GLA Population Projections,257384,2178.9,2178.9,118.1,21.8,11812.4
385,E09000008,Croydon,2020,GLA Population Projections,400272,8650.4,8650.4,46.3,86.5,4627.2
437,E09000009,Ealing,2020,GLA Population Projections,361833,5554.4,5554.4,65.1,55.5,6514.3
489,E09000010,Enfield,2020,GLA Population Projections,342993,8083.2,8220.1,42.4,80.8,4243.3


Choose important columns/rows and change column names

In [6]:
#clmns = ldn_df_raw.columns
#clmns
clmns = ['Code', 'Name', 'Population', 'Square_Kilometres']
ren_set = {'Name': 'Borough', 'Square_Kilometres': 'Square_km'}
ldn_df = (ldn_2020_df[clmns]
          .reset_index(drop=True)
          .rename(columns=ren_set)
          .drop([33, 34, 35])
         )
ldn_df

,Code,Borough,Population,Square_km
0,E09000001,City of London,8046,2.9
1,E09000002,Barking and Dagenham,218367,36.1
2,E09000003,Barnet,407148,86.7
3,E09000004,Bexley,254970,60.6
4,E09000005,Brent,343853,43.2
5,E09000006,Bromley,336395,150.1
6,E09000007,Camden,257384,21.8
7,E09000008,Croydon,400272,86.5
8,E09000009,Ealing,361833,55.5
9,E09000010,Enfield,342993,80.8


Use python geocoder library to get geograpical coordination

In [7]:
# get coordinates of London
ldn_address = 'London, UK'

geolocator = Nominatim(user_agent="london_explorer")
location = geolocator.geocode(ldn_address)
latitude = location.latitude
longitude = location.longitude
ldn_center = [latitude, longitude]
print('The geograpical coordinate of {} are {}, {}.'.format(ldn_address, latitude, longitude))

The geograpical coordinate of London, UK are 51.5073219, -0.1276474.


In [8]:
# define a function to get coordinates
def get_latlng(borough):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, {}'.format(borough,ldn_address))
        lat_lng_coords = g.latlng
    return lat_lng_coords

coords = [get_latlng(borough) for borough in ldn_df["Borough"].tolist()]

df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
ldn_df['Latitude'] = df_coords['Latitude']
ldn_df['Longitude'] = df_coords['Longitude']
#print("Geographical coordinates of five borough are as below")
ldn_df

,Code,Borough,Population,Square_km,Latitude,Longitude
0,E09000001,City of London,8046,2.9,51.520500,-0.097430
1,E09000002,Barking and Dagenham,218367,36.1,51.543932,0.133157
2,E09000003,Barnet,407148,86.7,51.527095,-0.066826
3,E09000004,Bexley,254970,60.6,51.452078,0.069931
4,E09000005,Brent,343853,43.2,51.609783,-0.194672
5,E09000006,Bromley,336395,150.1,51.601511,-0.066365
6,E09000007,Camden,257384,21.8,51.532360,-0.127960
7,E09000008,Croydon,400272,86.5,51.593480,-0.083420
8,E09000009,Ealing,361833,55.5,51.514060,-0.300730
9,E09000010,Enfield,342993,80.8,51.540024,-0.077502


Create a map of boroughs

In [9]:
ldn_map= folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng,  borough in zip(ldn_df['Latitude'], ldn_df['Longitude'], ldn_df['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(ldn_map)  
    
ldn_map

Set credentails for Foursquare API

In [10]:
CLIENT_ID = 'D3OBKIEVXNGYA21CL4RTOJ5BSKPZWUIHZL0O4K2STH0P1SFI' # your Foursquare ID
CLIENT_SECRET = '5HENKAN3MFKBJYMFN503N5AAKS1DH1MED2KLSPEMVLO4DKNU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: D3OBKIEVXNGYA21CL4RTOJ5BSKPZWUIHZL0O4K2STH0P1SFI
CLIENT_SECRET:5HENKAN3MFKBJYMFN503N5AAKS1DH1MED2KLSPEMVLO4DKNU


Explore the first neighborhod

In [11]:
# Get data of first neighborhood
borough_latitude = ldn_df['Latitude'][0] 
borough_longitude = ldn_df['Longitude'][0] 

borough_name = ldn_df['Borough'][0] 

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               borough_latitude, 
                                                               borough_longitude))

Latitude and longitude values of City of London are 51.52050000000003, -0.09742999999997437.


Find near by venues

In [12]:
# Setup API URL to explore venues near by
LIMIT = 100
radius = 1000
url = ('https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'
       .format(CLIENT_ID, CLIENT_SECRET, borough_latitude, borough_longitude, VERSION, radius, LIMIT))
borough_json = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = borough_json['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

<ipython-input-12-256435cc67a8>:22: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Barbican Centre,Performing Arts Venue,51.519855,-0.093942
1,Barbican Art Gallery,Art Gallery,51.519800,-0.093969
2,Virgin Active,Gym / Fitness Center,51.517952,-0.097651
3,Barbican Concert Hall,Concert Hall,51.519632,-0.093661
4,Barbican Theatre,Theater,51.519722,-0.093606
5,the Charterhouse London,Museum,51.520850,-0.099305
6,Dose Espresso,Coffee Shop,51.519553,-0.099406
7,St Bartholomew the Great (St Bartholomew-the-G...,Church,51.518631,-0.099890
8,Barbican Cinemas 2&3,Indie Movie Theater,51.521073,-0.092979
9,Giddy Up,Coffee Shop,51.522191,-0.093470


A function to repeat the same process to all boroughs

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Receive Venue Data from Foursquare for all boroughs

In [14]:
ldn_venues_df = getNearbyVenues(names=ldn_df['Borough'],
                                   latitudes=ldn_df['Latitude'],
                                   longitudes=ldn_df['Longitude']
                                  )

City of London
Barking and Dagenham
Barnet
Bexley
Brent
Bromley
Camden
Croydon
Ealing
Enfield
Greenwich
Hackney
Hammersmith and Fulham
Haringey
Harrow
Havering
Hillingdon
Hounslow
Islington
Kensington and Chelsea
Kingston upon Thames
Lambeth
Lewisham
Merton
Newham
Redbridge
Richmond upon Thames
Southwark
Sutton
Tower Hamlets
Waltham Forest
Wandsworth
Westminster


Some rows form resulting dataframe with venues

In [15]:
ldn_venues_df.head(12)

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,City of London,51.5205,-0.09743,Barbican Centre,51.519855,-0.093942,Performing Arts Venue
1,City of London,51.5205,-0.09743,Barbican Art Gallery,51.519800,-0.093969,Art Gallery
2,City of London,51.5205,-0.09743,Virgin Active,51.517952,-0.097651,Gym / Fitness Center
3,City of London,51.5205,-0.09743,Barbican Concert Hall,51.519632,-0.093661,Concert Hall
4,City of London,51.5205,-0.09743,Barbican Theatre,51.519722,-0.093606,Theater
5,City of London,51.5205,-0.09743,the Charterhouse London,51.520850,-0.099305,Museum
6,City of London,51.5205,-0.09743,Dose Espresso,51.519553,-0.099406,Coffee Shop
7,City of London,51.5205,-0.09743,St Bartholomew the Great (St Bartholomew-the-G...,51.518631,-0.099890,Church
8,City of London,51.5205,-0.09743,Barbican Cinemas 2&3,51.521073,-0.092979,Indie Movie Theater
9,City of London,51.5205,-0.09743,Giddy Up,51.522191,-0.093470,Coffee Shop


How many venues were returned for each neighborhood

In [16]:
ldn_venues_df.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Barking and Dagenham,6,6,6,6,6,6
Barnet,100,100,100,100,100,100
Bexley,21,21,21,21,21,21
Brent,38,38,38,38,38,38
Bromley,38,38,38,38,38,38
Camden,100,100,100,100,100,100
City of London,100,100,100,100,100,100
Croydon,43,43,43,43,43,43
Ealing,100,100,100,100,100,100


Unique categories can be curated from all the returned venues

In [17]:
print('There are {} uniques categories.'.format(len(ldn_venues_df['Venue Category'].unique())))

There are 278 uniques categories.


In [18]:
ldn_venues_df

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,City of London,51.520500,-0.097430,Barbican Centre,51.519855,-0.093942,Performing Arts Venue
1,City of London,51.520500,-0.097430,Barbican Art Gallery,51.519800,-0.093969,Art Gallery
2,City of London,51.520500,-0.097430,Virgin Active,51.517952,-0.097651,Gym / Fitness Center
3,City of London,51.520500,-0.097430,Barbican Concert Hall,51.519632,-0.093661,Concert Hall
4,City of London,51.520500,-0.097430,Barbican Theatre,51.519722,-0.093606,Theater
5,City of London,51.520500,-0.097430,the Charterhouse London,51.520850,-0.099305,Museum
6,City of London,51.520500,-0.097430,Dose Espresso,51.519553,-0.099406,Coffee Shop
7,City of London,51.520500,-0.097430,St Bartholomew the Great (St Bartholomew-the-G...,51.518631,-0.099890,Church
8,City of London,51.520500,-0.097430,Barbican Cinemas 2&3,51.521073,-0.092979,Indie Movie Theater
9,City of London,51.520500,-0.097430,Giddy Up,51.522191,-0.093470,Coffee Shop


In [19]:
print('Top 6 boroughs with restaurants and cafés:')
#ldn_venues_df_temp = ldn_venues_df.groupby(['Borough','Venue Category']).count()['Venue']
#ldn_venues_df_temp
#ldn_venues_df_temp2 = ldn_venues_df[ldn_venues_df['Venue Category'] > ]
mask = ldn_venues_df['Venue Category'].str.contains('Restaurant|Bar|Café|Pub', case=False, regex=True)
#ldn_venues_df_temp = ldn_venues_df[mask].groupby(['Borough','Venue Category']).count()['Venue']
ldn_venues_top6 = (ldn_venues_df[mask]
                      .groupby(['Borough'])
                      .count()['Venue']
                      .sort_values(ascending=False)
                      [:6]
                     )
ldn_venues_top6
#ldn_venues_df_temp.sort_values(ascending=False)
#ldn_venues_df_temp.sort()
#ldn_venues_df[mask]
                      

Top 6 boroughs with restaurants and cafés:


Borough
Hammersmith and Fulham    48
Enfield                   47
Lambeth                   46
Barnet                    45
Kensington and Chelsea    43
Ealing                    42
Name: Venue, dtype: int64

In [67]:
if True not in ldn_venues_df['Venue Category'].str.contains('Afghani', case=False, regex=True).tolist():
    print('No Afghani Restaruants')

No Afghani Restaruants


In [68]:
url = ('https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&q={}'
       .format(CLIENT_ID, CLIENT_SECRET, 51.5073219, -0.1276474, VERSION, radius, LIMIT, 'Русский'))
afghani_json = requests.get(url).json()

In [32]:
#Afghani_json
print(url)

https://api.foursquare.com/v2/venues/explore?client_id=D3OBKIEVXNGYA21CL4RTOJ5BSKPZWUIHZL0O4K2STH0P1SFI&client_secret=5HENKAN3MFKBJYMFN503N5AAKS1DH1MED2KLSPEMVLO4DKNU&ll=51.5073219,-0.1276474&v=20180605&radius=1000&limit=100&q=Русский


In [33]:
venues_rus = afghani_json["response"]['groups'][0]['items']

In [34]:
nearby_venues_afg = json_normalize(venues_afg)
nearby_venues_afg
#type(nearby_venues_rus)

<ipython-input-34-def0a07ea145>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues_rus = json_normalize(venues_rus)


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.venuePage.id,venue.location.crossStreet,venue.location.neighborhood
0,e-0-4ac518cdf964a520e6a520e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4ac518cdf964a520e6a520e3,National Gallery,Trafalgar Sq,51.508876,-0.128478,"[{'label': 'display', 'lat': 51.50887601013219...",182,WC2N 5DN,GB,London,Greater London,United Kingdom,"[Trafalgar Sq, London, Greater London, WC2N 5D...","[{'id': '4bf58dd8d48988d18f941735', 'name': 'A...",0,[],51313653,NaN,NaN
1,e-0-4ac518cef964a520f9a520e3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4ac518cef964a520f9a520e3,Trafalgar Square,Trafalgar Sq,51.507987,-0.128048,"[{'label': 'display', 'lat': 51.507987, 'lng':...",79,WC2N 5DX,GB,London,Greater London,United Kingdom,"[Trafalgar Sq, London, Greater London, WC2N 5D...","[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",0,[],NaN,NaN,NaN
2,e-0-50aa7c98e4b0d042f9f0bd59-2,0,"[{'summary': 'This spot is popular', 'type': '...",50aa7c98e4b0d042f9f0bd59,East Trafalgar Square Fountain,Trafalgar Sq,51.508088,-0.127700,"[{'label': 'display', 'lat': 51.50808805163309...",85,WC2N 5DN,GB,London,Greater London,United Kingdom,"[Trafalgar Sq, London, Greater London, WC2N 5D...","[{'id': '56aa371be4b08b9a8d573547', 'name': 'F...",0,[],NaN,NaN,NaN
3,e-0-4d274945d86aa09024d41dc0-3,0,"[{'summary': 'This spot is popular', 'type': '...",4d274945d86aa09024d41dc0,Corinthia Hotel,Whitehall Pl,51.506607,-0.124460,"[{'label': 'display', 'lat': 51.50660672347560...",234,SW1A 2BD,GB,London,Greater London,United Kingdom,"[Whitehall Pl, London, Greater London, SW1A 2B...","[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",0,[],NaN,NaN,NaN
4,e-0-4b9a4eb0f964a5202bab35e3-4,0,"[{'summary': 'This spot is popular', 'type': '...",4b9a4eb0f964a5202bab35e3,St Martin-in-the-Fields,5 Trafalgar Sq,51.508746,-0.126507,"[{'label': 'display', 'lat': 51.50874574282390...",177,WC2N 4JJ,GB,London,Greater London,United Kingdom,"[5 Trafalgar Sq, London, Greater London, WC2N ...","[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",0,[],NaN,NaN,NaN
5,e-0-50aa6e60e4b078585157bbb5-5,0,"[{'summary': 'This spot is popular', 'type': '...",50aa6e60e4b078585157bbb5,Trafalgar Square Lions,Trafalgar Sq,51.507641,-0.127888,"[{'label': 'display', 'lat': 51.50764064540292...",39,WC2N 5DN,GB,London,Greater London,United Kingdom,"[Trafalgar Sq, London, Greater London, WC2N 5D...","[{'id': '52e81612bcbc57f1066b79ed', 'name': 'O...",0,[],NaN,NaN,NaN
6,e-0-4ae5c61ff964a520efa121e3-6,0,"[{'summary': 'This spot is popular', 'type': '...",4ae5c61ff964a520efa121e3,Nelson's Column,Trafalgar Sq,51.507744,-0.127931,"[{'label': 'display', 'lat': 51.50774415021993...",50,WC2N 5DN,GB,London,Greater London,United Kingdom,"[Trafalgar Sq, London, Greater London, WC2N 5D...","[{'id': '4bf58dd8d48988d12d941735', 'name': 'M...",0,[],NaN,NaN,NaN
7,e-0-4acdb591f964a520c7cc20e3-7,0,"[{'summary': 'This spot is popular', 'type': '...",4acdb591f964a520c7cc20e3,Gordon's Wine Bar,47 Villiers St,51.507911,-0.123293,"[{'label': 'display', 'lat': 51.5079110929777,...",308,WC2N 6NE,GB,London,Greater London,United Kingdom,"[47 Villiers St (Watergate Walk), London, Grea...","[{'id': '4bf58dd8d48988d123941735', 'name': 'W...",0,[],NaN,Watergate Walk,NaN
8,e-0-4ac518cdf964a520f0a520e3-8,0,"[{'summary': 'This spot is popular', 'type': '...",4ac518cdf964a520f0a520e3,Horse Guards Parade,The Household Division,51.504847,-0.126590,"[{'label': 'display', 'lat': 51.50484733116023...",285,SW1A 2AX,GB,London,Greater London,United Kingdom,"[The Household Division (Horse Guards Rd), Lon...","[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",0,[],NaN,Horse Guards Rd,

In [66]:
clmn_res_afg = ['Name', 'Borough', 'Rating']
res_afg_df = pd.DataFrame(columns=clmn_res_afg)
res_afg_df['Name'] = ['Kabul_Food', 'Kolba', 'Palaw Resturant', 'Zam Zam', 'Balkh_Resturant']
res_afg_df['Borough'] = ['Kensington and Chelsea', 'Kensington and Chelsea', 'Kensington and Chelsea', 'Westminster', 'Islington']
res_afg_df['Rating'] = [8, 8, 10, 8, 8]
res_afg_df

,Name,Borough,Rating
0,Kabul_Food,Kensington and Chelsea,8
1,Kolba,Kensington and Chelsea,8
2,Palaw Resturant,Kensington and Chelsea,10
3,Zam Zam,Westminster,8
4,Balkh_Resturant,Islington,8
